In [9]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import zipfile
import numpy as np
tf.__version__

'2.0.0'

In [2]:
def load_data_jay_lyrics():
    """Load the Jay Chou lyric data set (available in the Chinese book)."""
    with zipfile.ZipFile('../../data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

corpus_indices, char_to_idx, idx_to_char, vocab_size = load_data_jay_lyrics()
vocab_size

1027

## 算法

LSTM 中引入了3个门，即输入门（input gate）、遗忘门（forget gate）和输出门（output gate），以及与隐藏状态形状相同的记忆细胞（某些文献把记忆细胞当成一种特殊的隐藏状态），从而记录额外的信息。

### 1. 输入门、遗忘门和输出门

与门控循环单元中的重置门和更新门一样,长短期记忆的门的输入均为当前时间步输入$\boldsymbol{X}_t$与上一时间步隐藏状态$\boldsymbol{H}_{t-1}$，输出由激活函数为sigmoid函数的全连接层计算得到。这3个门元素的值域均为 $[0,1]$ 。

具体来说，假设隐藏单元个数为$h$，给定时间步$t$的小批量输入$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$（样本数为$n$，输入个数为$d$）和上一时间步隐藏状态$\boldsymbol{H}_{t-1} \in \mathbb{R}^{n \times h}$。 时间步$t$的输入门$\boldsymbol{I}_t \in \mathbb{R}^{n \times h}$、遗忘门$\boldsymbol{F}_t \in \mathbb{R}^{n \times h}$和输出门$\boldsymbol{O}_t \in \mathbb{R}^{n \times h}$分别计算如下：

$$\begin{split}\begin{aligned}
\boldsymbol{I}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xi} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hi} + \boldsymbol{b}_i),\\
\boldsymbol{F}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xf} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hf} + \boldsymbol{b}_f),\\
\boldsymbol{O}_t &= \sigma(\boldsymbol{X}_t \boldsymbol{W}_{xo} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{ho} + \boldsymbol{b}_o),
\end{aligned}\end{split}$$

其中的$\boldsymbol{W}_{xi}, \boldsymbol{W}_{xf}, \boldsymbol{W}_{xo} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hi}, \boldsymbol{W}_{hf}, \boldsymbol{W}_{ho} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_i, \boldsymbol{b}_f, \boldsymbol{b}_o \in \mathbb{R}^{1 \times h}$是偏差参数。

### 2.候选记忆细胞

长短期记忆需要计算候选记忆细胞$\tilde{\boldsymbol{C}}_t$。它的计算与上面介绍的3个门类似，但使用了值域在$[−1,1]$的tanh函数作为激活函数

具体来说，时间步$t$的候选记忆细胞$\tilde{\boldsymbol{C}}_t \in \mathbb{R}^{n \times h}$的计算为
$$\tilde{\boldsymbol{C}}_t = \text{tanh}(\boldsymbol{X}_t \boldsymbol{W}_{xc} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hc} + \boldsymbol{b}_c),$$

其中$\boldsymbol{W}_{xc} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hc} \in \mathbb{R}^{h \times h}$是权重参数，$\boldsymbol{b}_c \in \mathbb{R}^{1 \times h}$是偏差参数。

### 3.记忆细胞

我们可以通过元素值域在$[0,1]$的输入门、遗忘门和输出门来控制隐藏状态中信息的流动，这一般也是通过使用按元素乘法（符号为 ⊙ ）来实现的。当前时间步记忆细胞$\boldsymbol{C}_t \in \mathbb{R}^{n \times h}$的计算组合了上一时间步记忆细胞和当前时间步候选记忆细胞的信息，并通过遗忘门和输入门来控制信息的流动：
$$\boldsymbol{C}_t = \boldsymbol{F}_t \odot \boldsymbol{C}_{t-1} + \boldsymbol{I}_t \odot \tilde{\boldsymbol{C}}_t.$$

### 4.隐藏状态
有了记忆细胞以后，接下来我们还可以通过输出门来控制从记忆细胞到隐藏状态$\boldsymbol{H}_t \in \mathbb{R}^{n \times h}$的信息的流动：
$$\boldsymbol{H}_t = \boldsymbol{O}_t \odot \text{tanh}(\boldsymbol{C}_t).$$
这里的tanh函数确保隐藏状态元素值在-1到1之间。需要注意的是，当输出门近似1时，记忆细胞信息将传递到隐藏状态供输出层使用；当输出门近似0时，记忆细胞信息只自己保留。下图展示了长短期记忆中隐藏状态的计算。

![image](http://zh.d2l.ai/_images/lstm_3.svg)

### 5.最终输出

$$\boldsymbol{Y}_t = \boldsymbol{H}_t \boldsymbol{W}_{hq} + \boldsymbol{b}_q.$$

In [5]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

In [27]:
def lstm(inputs,states,params):
    w_xi,w_hi,b_i,w_xf,w_hf,b_f,w_xo,w_ho,b_o,w_xc,w_hc,b_c,w_hq,b_q = params
    H,C = states
    outputs = []
    for x in inputs:
        #忘记门
        f = keras.activations.sigmoid(tf.matmul(x,w_xf) + tf.matmul(H,w_hf) + b_f)
        # 输入门
        i = keras.activations.sigmoid(tf.matmul(x,w_xi) + tf.matmul(H,w_hi) + b_i)
        # 输出门
        o = keras.activations.sigmoid(tf.matmul(x,w_xo) + tf.matmul(H,w_ho) + b_o)
        # 候选记忆细胞
        c_hat = keras.activations.tanh(tf.matmul(x,w_xc) + tf.matmul(H,w_hc) + b_c)
        # 记忆细胞
        C = tf.multiply(f,C) + tf.multiply(i,c_hat)
        # 隐藏层
        H = tf.multiply(o,keras.activations.tanh(C))
        y = tf.matmul(H,w_hq) + b_q
        outputs.append(y)
    return tf.stack(outputs),(H,C)
        

In [28]:
def init_lstm_states(batch_size,num_hiddens):
    return (
        tf.zeros(shape=(batch_size,num_hiddens)),
        tf.zeros(shape=(batch_size,num_hiddens))
    )

In [29]:
def init_params():
    
    def _threes():
        return (tf.random.normal(shape=(num_inputs,num_hiddens)),tf.random.normal(shape=(num_hiddens,num_hiddens)),tf.zeros(shape=(1,num_hiddens)))
    
    w_xi,w_hi,b_i = _threes()
    w_xf,w_hf,b_f = _threes()
    w_xo,w_ho,b_o = _threes()
    w_xc,w_hc,b_c = _threes()
    w_hq,b_q = tf.random.normal(shape=(num_hiddens,num_outputs)),tf.zeros(shape=(1,num_outputs))
    
    return w_xi,w_hi,b_i,w_xf,w_hf,b_f,w_xo,w_ho,b_o,w_xc,w_hc,b_c,w_hq,b_q

In [30]:
def to_onehot(X,depth):
    outs = [tf.one_hot(x,depth=depth) for x in tf.transpose(X)]
    return tf.stack(outs,axis=0)

In [31]:
x = np.array(range(5)).reshape((1,5))
inputs = to_onehot(x,vocab_size)

In [32]:
state = init_lstm_states(x.shape[0],num_hiddens)
lstm(inputs,state,init_params())

(<tf.Tensor: id=1294, shape=(5, 1, 1027), dtype=float32, numpy=
 array([[[  0.07173868,   1.6845028 ,   5.2634587 , ...,  -0.09351012,
            2.270501  ,   1.3396002 ]],
 
        [[  7.1314826 ,  -0.95359766,   5.609708  , ...,  -7.7840066 ,
           10.76433   ,   0.508923  ]],
 
        [[  1.6182783 ,  -1.1211449 ,  12.420524  , ...,  -7.902752  ,
            7.1471243 ,  -7.1611023 ]],
 
        [[  7.848685  ,   0.68195987,   5.4883237 , ...,   3.5423014 ,
           -2.8229854 , -12.434613  ]],
 
        [[ -4.4162593 ,   9.665938  ,   7.026255  , ...,  13.074351  ,
           -2.5717328 ,   0.456568  ]]], dtype=float32)>,
 (<tf.Tensor: id=1291, shape=(1, 256), dtype=float32, numpy=
  array([[-4.52327952e-02, -6.80377185e-01, -1.22630297e-06,
          -8.65447223e-01,  2.64880329e-01,  1.59439128e-02,
          -9.01117772e-02, -1.64594792e-03, -7.25754499e-01,
          -8.60191360e-02,  3.62363756e-01,  7.82752689e-03,
           2.13399503e-08,  2.97417641e-01,  6.577

In [33]:
keras_lstm = layers.LSTM(num_outputs)

In [34]:
keras_lstm(inputs)

<tf.Tensor: id=1437, shape=(5, 1027), dtype=float32, numpy=
array([[-0.00398049,  0.00267648, -0.00488786, ..., -0.00096839,
         0.00080001,  0.00018608],
       [ 0.00772672,  0.00621579, -0.00172489, ...,  0.0075411 ,
        -0.00592683, -0.00439078],
       [-0.00638593, -0.00090984,  0.00036399, ...,  0.00024921,
         0.00673251,  0.00221328],
       [-0.00393665, -0.00730946, -0.00665093, ..., -0.00705277,
         0.00806974,  0.00484846],
       [-0.00638531, -0.00075025, -0.00379624, ..., -0.00266368,
         0.00155018, -0.00622312]], dtype=float32)>